# Pontifícia Universidade Católica do Paraná
## Disciplina: Técnicas de Machine Learning
#### Conteúdo complementar da Semana 6 - Pipelines e Otimização de Hiperparâmetros

A ideia deste notebook é a de mostrar o uso de pipelines e da otimização de hiperparâmetros para termos modelos que com um fluxo completo de transformação de dados (no caso dos pipelines) e que tenham a melhor performance possível (no caso da otimização dos hiperparâmetros). Aqui, entenda que *hiperparâmetros* são aqueles parâmetros que ficam dentro dos modelos de aprendizagem supervisionada como, por exemplo, o ```n_neighbors``` no KNN e ```n_estimators``` do random forest.

Da mesma forma que comentamos nos notebooks anteriores aproveite este notebook para entender as funções e, quando tiver alguma dúvida sobre o significado de algum novo parâmetro, procurar na documentação oficial das bibliotecas para entender melhor quais são as possibilidades. Exemplo: <em>"o que significa o</em> ```sep``` <em>no pd.read.csv?"</em>

In [3]:
import pandas as pd # importando o pandas para manipularmos os datasets

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV # utilizado para o split entre treinamento e teste e para a otimização de hiperparâmetros
from sklearn.ensemble import * # importando vários ensembles para que possamos testá-los posteriormente
from sklearn.pipeline import Pipeline # utilizado para criar pipelines
from sklearn.neighbors import KNeighborsClassifier # utilizado para treinar o KNN
from sklearn.linear_model import LogisticRegression # utilizado para treinar um modelo de classificação (regressão logística - apesar do nome é para problemas de classificação)
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, RobustScaler # utilizado para converter colunas do tipo texto para numéricas
from sklearn.decomposition import PCA # utilizado para fazer o PCA nos pipelines
from sklearn.metrics import f1_score, make_scorer # utilizado para calcular a performance dos pipelines

from lightgbm import LGBMClassifier # utilizado para treinar o LightGBM

from sklearn import set_config # utilizado para mostrar os passos do pipeline de forma visual
set_config(display='diagram') # forçando para que os passos do pipeline sejam mostrados em visual

Utilizaremos o <a href="http://archive.ics.uci.edu/ml/datasets/Adult">adult dataset</a>. Ele contém os dados do censo populacional feito nos EUA em 1994. Traduzimos os nomes dos atributos para um melhor entendimento. O significado deles é:

- *idade*: a idade da pessoa.
- *classetrabalhadora*: a classe trabalhadora na qual aquela pessoa está inserida. Exemplos incluem: funcionário público municipal/estadual/federal; nunca trabalhou; funcionário autônomo; setor privado; sem renda.
- *totalpessoasestimadas*: a quantidade de habitantes no qual acredita-se que esta pessoa representa. Lembre-se que o censo pode ser de uma **amostra** de pessoas. Como é uma amostra vale saber quantas pessoas ao todo uma determinada instância representaria.
- *escolaridade*: o nível de escolaridade da pessoa.
- *escolaridadenum*: o mesmo nível, mas convertido para uma escala numérica (similar ao uso do LabelEncoder).
- *estadocivil*: o estado civil da pessoa. Exemplos incluem: casado, solteiro, divorciado, viúvo.
- *cargo*: o cargo da pessoa. Exemplos incluem: suporte técnico, vendas, gerencial, forças armadas, limpeza, transportes.
- *relacionamento*: o papel da pessoa na família. Exemplos incluem: esposa, esposo, não-casado, filho(a), não está em família.
- *raca*: a raça da pessoa.
- *sexo*: o sexo da pessoa.
- *ganhocapital* e *perdacapital*: ganhos e perdas de capital da pessoa.
- *horasporsemana*: horas trabalhadas por semana.
- *pais*: país de origem.
- *rendaacima50k*: é a nossa **classe**. Queremos prever se uma pessoa ganha abaixo ou acima de 50K dólares. Os valores incluem "<=50K" e ">50K".

Observe que vários dados não estão em uma escala numérica. Se tentarmos rodar um ```RandomForestClassifier``` ou qualquer outro modelo teremos um erro. Logo, precisaremos primeiro tratar estas colunas antes de criar um algoritmo de aprendizagem supervisionada.

In [5]:
df_adult = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
                       names=['idade', 'classetrabalhadora', 'totalpessoasestimadas', 'escolaridade',
                              'escolaridadenum', 'estadocivil', 'cargo', 'relacionamento', 'raca', 
                              'sexo', 'ganhoscapital', 'perdascapital', 'horastrabalhadasporsemana',
                              'pais', 'renda'],
                          index_col=False)

df_adult

,idade,classetrabalhadora,totalpessoasestimadas,escolaridade,escolaridadenum,estadocivil,cargo,relacionamento,raca,sexo,ganhoscapital,perdascapital,horastrabalhadasporsemana,pais,renda
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [6]:
# criando uma cópia do dataset para fins de teste
df_adult_copy = df_adult.copy()

# removendo o "escolaridade" já que temos o "escolaridadenum"
df_adult_copy = df_adult_copy.drop('escolaridade', axis=1)

# split entre treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df_adult_copy.drop('renda', axis=1), # aqui informamos os atributos
                                                    df_adult_copy['renda'], # aqui informamos as labels e na mesma ordem dos atributos
                                                    test_size=0.20, # informamos a porcentagem de divisão da base. Geralmente é algo entre 20% (0.20) a 35% (0.35)
                                                    random_state=0) # aqui informamos um "seed". É um valor aleatório e usado para que alguns algoritmos iniciem de forma aleatória a sua divisão.

In [7]:
# convertendo todas as colunas que são do tipo texto
# o handle_unknown é usado porque podem existir valores que só existem no X_test e não no X_train
# e queremos que a conversão não retorne em um erro caso isto aconteça
encoder_df = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1).fit(X_train)

# convertendo X_train e X_test para valores numéricos.
# não usamos o fit_transform porque precisaremos dos encoders novamente para X_test e y_test
X_train = encoder_df.transform(X_train)
X_train

array([[1.9000e+01, 4.0000e+00, 8.4360e+03, ..., 0.0000e+00, 3.9000e+01,
        3.9000e+01],
       [1.8000e+01, 4.0000e+00, 1.0730e+04, ..., 0.0000e+00, 5.3000e+01,
        3.9000e+01],
       [2.1000e+01, 4.0000e+00, 1.4340e+03, ..., 0.0000e+00, 4.9000e+01,
        3.9000e+01],
       ...,
       [6.0000e+00, 4.0000e+00, 1.1240e+03, ..., 0.0000e+00, 3.9000e+01,
        3.9000e+01],
       [2.8000e+01, 2.0000e+00, 1.2074e+04, ..., 0.0000e+00, 4.4000e+01,
        3.9000e+01],
       [8.0000e+00, 4.0000e+00, 9.6630e+03, ..., 0.0000e+00, 4.7000e+01,
        3.9000e+01]])

In [8]:
# agora, treinamos um algoritmo - usaremos o KNN como teste
clf = KNeighborsClassifier().fit(X_train, y_train)

In [9]:
# prevendo os resultados para o X_test
# note que precisaremos chamar os encoders novamente já que treinamos o X_train e y_train contendo somente números
X_test = encoder_df.transform(X_test)
y_pred = clf.predict(X_test)
display(f'Resultados de y_pred: {y_pred}')

"Resultados de y_pred: [' <=50K' ' <=50K' ' <=50K' ... ' >50K' ' <=50K' ' >50K']"

# Pipelines

Percebeu a quantidade de passos necessários para treinar e chegar aos resultados finais? Vamos usar agora o pipeline para fazer a **mesma** coisa. Primeiramente, vamos recriar o X_train/X_test/y_train/y_test porque no passo anterior havíamos os convertido para uma escala numérica.

In [11]:
# criando uma cópia do dataset para fins de teste
df_adult_copy = df_adult.copy()

# removendo o "escolaridade" já que temos o "escolaridadenum"
df_adult_copy = df_adult_copy.drop('escolaridade', axis=1)

# split entre treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df_adult_copy.drop('renda', axis=1), # aqui informamos os atributos
                                                    df_adult_copy['renda'], # aqui informamos as labels e na mesma ordem dos atributos
                                                    test_size=0.20, # informamos a porcentagem de divisão da base. Geralmente é algo entre 20% (0.20) a 35% (0.35)
                                                    random_state=0) # aqui informamos um "seed". É um valor aleatório e usado para que alguns algoritmos iniciem de forma aleatória a sua divisão.

Agora sim, vamos para o pipeline: compare com o código anterior. Note que o encoder e o KNN ficam encapsulados dentro de um único pipeline. Fazemos **só um** ```fit``` e só um ```predict```.

Observe que o pipeline sempre segue a mesma forma:
1. Estabelecemos dentro do ```Pipeline``` uma lista de passos.
2. Dentro dessa lista colocamos a **ordem** de execução: no nosso exemplo, primeiro precisamos fazer a conversão dos dados em texto para número. Só depois disso que treinaremos o KNN.
3. Dentro da lista adicionamos uma **tupla** (este conjunto de itens entre parênteses): uma tupla é um conjunto de um ou mais itens. Aqui, o primeiro item é sempre um nome qualquer que atribuímos a um passo e o segundo item é sempre a classe que queremos utilizar, como o ```OrdinalEncoder``` e o ```KNeighborsClassifier```.
4. Neste pipeline, temos **dois** passos: o primeiro passo se chama *encoder* e utiliza o ```OrdinalEncoder```. O segundo passo se chama *modelo* e chama o ```KNeighborsClassifier```. Depois de definir os passos treinamos o pipeline por inteiro utilizando o ```fit```.

In [13]:
pipe = Pipeline([('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
                 ('modelo', KNeighborsClassifier())]).fit(X_train, y_train)

y_pred = pipe.predict(X_test)
display(f'Resultados de y_pred: {y_pred}')

display(f'Passos do pipeline (para fins ilustrativos):')
pipe

"Resultados de y_pred: [' <=50K' ' <=50K' ' <=50K' ... ' >50K' ' <=50K' ' >50K']"

'Passos do pipeline (para fins ilustrativos):'

Pipeline(steps=[('encoder',
                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                unknown_value=-1)),
                ('modelo', KNeighborsClassifier())])

Note que também podemos testar outras combinações de pipelines e/ou incluir novos passos. Exemplos:

In [15]:
pipe_pca = Pipeline([('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
                     ('pca', PCA(n_components=3)),
                     ('modelo', RandomForestClassifier())]).fit(X_train, y_train)

y_pred_pca = pipe_pca.predict(X_test)
display(f'Resultados de y_pred: {y_pred_pca}')

display(f'Passos do pipeline (para fins ilustrativos):')
pipe_pca

"Resultados de y_pred: [' <=50K' ' <=50K' ' <=50K' ... ' >50K' ' >50K' ' >50K']"

'Passos do pipeline (para fins ilustrativos):'

Pipeline(steps=[('encoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
                ('pca', PCA(n_components=3)),
                ('modelo', RandomForestClassifier())])

In [16]:
pipe_rf = Pipeline([('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
                    ('modelo', RandomForestClassifier(random_state=0))]).fit(X_train, y_train)

y_pred_rf = pipe_rf.predict(X_test)
display(f'Resultados de y_pred: {y_pred_rf}')

display(f'Passos do pipeline (para fins ilustrativos):')
pipe_rf

"Resultados de y_pred: [' <=50K' ' <=50K' ' <=50K' ... ' >50K' ' <=50K' ' >50K']"

'Passos do pipeline (para fins ilustrativos):'

Pipeline(steps=[('encoder',
                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                unknown_value=-1)),
                ('modelo', RandomForestClassifier(random_state=0))])

In [17]:
pipe_scaler = Pipeline([('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
                        ('scaler', RobustScaler()),
                        ('modelo', KNeighborsClassifier())]).fit(X_train, y_train)

y_pred_scaler = pipe_scaler.predict(X_test)
display(f'Resultados de y_pred_scaler: {y_pred_scaler}')

display(f'Passos do pipeline (para fins ilustrativos):')
pipe_scaler

"Resultados de y_pred_scaler: [' <=50K' ' <=50K' ' <=50K' ... ' >50K' ' <=50K' ' >50K']"

'Passos do pipeline (para fins ilustrativos):'

Pipeline(steps=[('encoder',
                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                unknown_value=-1)),
                ('scaler', RobustScaler()),
                ('modelo', KNeighborsClassifier())])

In [18]:
pipe_lgbm = Pipeline([('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
                    ('modelo', LGBMClassifier(random_state=0))]).fit(X_train, y_train)

y_pred_lgbm = pipe_lgbm.predict(X_test)
display(f'Resultados de y_pred: {y_pred_lgbm}')

display(f'Passos do pipeline (para fins ilustrativos):')
pipe_lgbm

[LightGBM] [Info] Number of positive: 6246, number of negative: 19802
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006463 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 671
[LightGBM] [Info] Number of data points in the train set: 26048, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.239788 -> initscore=-1.153842
[LightGBM] [Info] Start training from score -1.153842


"Resultados de y_pred: [' <=50K' ' <=50K' ' <=50K' ... ' >50K' ' <=50K' ' >50K']"

'Passos do pipeline (para fins ilustrativos):'

Pipeline(steps=[('encoder',
                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                unknown_value=-1)),
                ('modelo', LGBMClassifier(random_state=0))])

## Comparando os resultados

Utilizaremos o F1 score para comparar os resultados. Observe o impacto das diferentes configurações de pipelines sobre os resultados.

In [20]:
display(f"Resultados para o primeiro pipeline: {f1_score(y_test, y_pred, pos_label=' >50K')}")
display(f"Resultados para o segundo pipeline: {f1_score(y_test, y_pred_pca, pos_label=' >50K')}")
display(f"Resultados para o terceiro pipeline: {f1_score(y_test, y_pred_rf, pos_label=' >50K')}")
display(f"Resultados para o quarto pipeline: {f1_score(y_test, y_pred_scaler, pos_label=' >50K')}")
display(f"Resultados para o quinto pipeline: {f1_score(y_test, y_pred_lgbm, pos_label=' >50K')}")

'Resultados para o primeiro pipeline: 0.46098596147567744'

'Resultados para o segundo pipeline: 0.573489010989011'

'Resultados para o terceiro pipeline: 0.6677989130434783'

'Resultados para o quarto pipeline: 0.6459459459459459'

'Resultados para o quinto pipeline: 0.7068565750248427'

# Otimização de hiperparâmetros

A princípio o pipeline (```OrdinalEncoder``` + ```KNeighborsClassifier```) teve piores resultados. Mas será que conseguiríamos resultados melhores com ele? Existem dois principais métodos no scikit-learn para isso: o <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV">RandomSearchCV</a> e o <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV">GridSearchCV</a>.

O GridSearchCV é o que chamamos de *busca exaustiva*: ele testará todas as combinações de todos os parâmetros que você informar, o que pode ser bem lento dependendo de quantas combinações existirem.

O RandomSearchCV seleciona aleatoriamente algumas das combinações. É mais rápido, mas não significa necessariamente que encontrará os melhores resultados.

Existem outros algoritmos de otimização de hiperparâmetros como, por exemplo, a otimização bayesiana e o halving grid search. Por outro lado, o random search e o grid search são de longe os mais conhecidos. Para fazer a otimização precisamos informar todas as combinações que gostaríamos que fossem testadas. Dessa forma:

1. Abrimos a documentação do modelo que queremos otimizar (no caso, o <a href="https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier">KNeighborsClassifier</a>)
2. Escolhemos quais parâmetros gostaríamos de testar (exemplos: ```n_neighbors```, ```metric```, ```weights``` e ```leaf_size```).
3. Escolhemos quais valores testaríamos para cada uma das combinações inserindo-as em um formato de lista.
4. Executamos o otimizador para o modelo que queremos otimizar.

Observe o funcionamento abaixo. Se possível, teste também diferentes configurações.

In [22]:
# rodando o encoder antes de otimizar os hiperparâmetros
# se tentássemos otimizar *dentro* do pipeline teríamos um erro.
X_train_encoded = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1).fit_transform(X_train)

In [23]:
parametros = {
    'n_neighbors': [2, 5, 10, 20],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
    'weights': ['uniform', 'distance'],
    'leaf_size': [10, 30, 50]
}

Observe que o random search executará mais rápido, mas testará menos alternativas. Observe também que certas combinações são mais rápidas do que outras, e que o score também pode mudar radicalmente dependendo das combinações.

Finalmente, o "CV" significa cross validation, ou validação cruzada. De uma forma bem simplista ele divide a base de treinamento em cinco partes, treinando com quatro partes e avaliando com a quinta e alternando também entre todas as cinco partes. Se tiver interesse <a href="https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation">você pode aprender mais sobre isto aqui</a>.

In [25]:
# precisaremos encontrar os melhores parâmetros deste modelo.
# o parâmetro verbose mostra o passo-a-passo da execução do random search 
# o parâmetro scoring força que gostaríamos de maximizar o F1 score. Também poderíamos usar outros critérios
# lembre que o máximo que conseguimos até agora foi um score de 0.4603
clf_random = RandomizedSearchCV(KNeighborsClassifier(), parametros, verbose=3, scoring=make_scorer(f1_score, pos_label=' >50K'))
clf_random.fit(X_train_encoded, y_train)

# mostrando o melhor modelo segundo o Grid Search
best_random_search = clf_random.best_estimator_
display('Melhor classificador segundo o Random Search: ')
display(best_random_search)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END leaf_size=30, metric=euclidean, n_neighbors=20, weights=uniform;, score=0.275 total time=   0.4s
[CV 2/5] END leaf_size=30, metric=euclidean, n_neighbors=20, weights=uniform;, score=0.278 total time=   0.4s
[CV 3/5] END leaf_size=30, metric=euclidean, n_neighbors=20, weights=uniform;, score=0.268 total time=   0.4s
[CV 4/5] END leaf_size=30, metric=euclidean, n_neighbors=20, weights=uniform;, score=0.282 total time=   0.4s
[CV 5/5] END leaf_size=30, metric=euclidean, n_neighbors=20, weights=uniform;, score=0.239 total time=   0.5s
[CV 1/5] END leaf_size=10, metric=manhattan, n_neighbors=5, weights=uniform;, score=0.482 total time=   0.5s
[CV 2/5] END leaf_size=10, metric=manhattan, n_neighbors=5, weights=uniform;, score=0.483 total time=   0.4s
[CV 3/5] END leaf_size=10, metric=manhattan, n_neighbors=5, weights=uniform;, score=0.449 total time=   0.5s
[CV 4/5] END leaf_size=10, metric=manhattan, n_neighbors=5, we

'Melhor classificador segundo o Random Search: '

KNeighborsClassifier(leaf_size=10, metric='manhattan')

In [26]:
# precisaremos encontrar os melhores parâmetros deste modelo.
# o parâmetro verbose mostra o passo-a-passo da execução do grid search 
# o parâmetro scoring força que gostaríamos de maximizar o F1 score. Também poderíamos usar outros critérios
# lembre que o máximo que conseguimos até agora foi um score de 0.4603
clf_grid = GridSearchCV(KNeighborsClassifier(), parametros, verbose=3, scoring=make_scorer(f1_score, pos_label=' >50K'))
clf_grid.fit(X_train_encoded, y_train)

# mostrando o melhor modelo segundo o Grid Search
best_grid_search = clf_grid.best_estimator_
display('Melhor classificador segundo o Grid Search: ')
display(best_grid_search)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV 1/5] END leaf_size=10, metric=euclidean, n_neighbors=2, weights=uniform;, score=0.374 total time=   0.4s
[CV 2/5] END leaf_size=10, metric=euclidean, n_neighbors=2, weights=uniform;, score=0.367 total time=   0.4s
[CV 3/5] END leaf_size=10, metric=euclidean, n_neighbors=2, weights=uniform;, score=0.334 total time=   0.4s
[CV 4/5] END leaf_size=10, metric=euclidean, n_neighbors=2, weights=uniform;, score=0.396 total time=   0.5s
[CV 5/5] END leaf_size=10, metric=euclidean, n_neighbors=2, weights=uniform;, score=0.362 total time=   0.4s
[CV 1/5] END leaf_size=10, metric=euclidean, n_neighbors=2, weights=distance;, score=0.457 total time=   0.1s
[CV 2/5] END leaf_size=10, metric=euclidean, n_neighbors=2, weights=distance;, score=0.456 total time=   0.1s
[CV 3/5] END leaf_size=10, metric=euclidean, n_neighbors=2, weights=distance;, score=0.436 total time=   0.1s
[CV 4/5] END leaf_size=10, metric=euclidean, n_neighbors=2, wei

'Melhor classificador segundo o Grid Search: '

KNeighborsClassifier(leaf_size=10, metric='manhattan', weights='distance')

Em alguns casos o random search poderá chegar à mesma conclusão do grid search, mas de uma forma bem mais rápida. Por outro lado, não podemos **garantir** que o random search sempre chegará à melhor solução. Neste **caso em específico** ambos não encontraram uma mesma combinação do ```KNeighborsClassifier```.

Finalmente, vamos comparar os resultados:

In [28]:
pipe_original = Pipeline([('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
                          ('modelo', KNeighborsClassifier())]).fit(X_train, y_train)

y_pred_original = pipe_original.predict(X_test)
display(f'Resultados de y_pred: {y_pred_original}')

display(f'Passos do pipeline (para fins ilustrativos):')
pipe_original

"Resultados de y_pred: [' <=50K' ' <=50K' ' <=50K' ... ' >50K' ' <=50K' ' >50K']"

'Passos do pipeline (para fins ilustrativos):'

Pipeline(steps=[('encoder',
                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                unknown_value=-1)),
                ('modelo', KNeighborsClassifier())])

In [29]:
pipe_otimizado_randomsearch = Pipeline([('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
                                        ('modelo', best_random_search)]).fit(X_train, y_train)

y_pred_otimizado_randomsearch = pipe_otimizado_randomsearch.predict(X_test)
display(f'Resultados de y_pred: {y_pred_otimizado_randomsearch}')

display(f'Passos do pipeline (para fins ilustrativos):')
pipe_otimizado_randomsearch

"Resultados de y_pred: [' <=50K' ' <=50K' ' <=50K' ... ' >50K' ' <=50K' ' >50K']"

'Passos do pipeline (para fins ilustrativos):'

Pipeline(steps=[('encoder',
                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                unknown_value=-1)),
                ('modelo',
                 KNeighborsClassifier(leaf_size=10, metric='manhattan'))])

In [30]:
pipe_otimizado_gridsearch = Pipeline([('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
                                      ('modelo', best_grid_search)]).fit(X_train, y_train)

y_pred_otimizado_gridsearch = pipe_otimizado_gridsearch.predict(X_test)
display(f'Resultados de y_pred: {y_pred_otimizado_gridsearch}')

display(f'Passos do pipeline (para fins ilustrativos):')
pipe_otimizado_gridsearch

"Resultados de y_pred: [' <=50K' ' <=50K' ' <=50K' ... ' >50K' ' <=50K' ' >50K']"

'Passos do pipeline (para fins ilustrativos):'

Pipeline(steps=[('encoder',
                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                unknown_value=-1)),
                ('modelo',
                 KNeighborsClassifier(leaf_size=10, metric='manhattan',
                                      weights='distance'))])

In [31]:
display(f"Resultados para o pipeline original: {f1_score(y_test, y_pred_original, pos_label=' >50K')}")
display(f"Resultados para o pipeline otimizado via RandomSearchCV: {f1_score(y_test, y_pred_otimizado_randomsearch, pos_label=' >50K')}")
display(f"Resultados para o pipeline otimizado via GridSearchCV: {f1_score(y_test, y_pred_otimizado_gridsearch, pos_label=' >50K')}")

'Resultados para o pipeline original: 0.46098596147567744'

'Resultados para o pipeline otimizado via RandomSearchCV: 0.5076820307281229'

'Resultados para o pipeline otimizado via GridSearchCV: 0.5192936559843034'

Observe que foi possível melhorar a performance. Para este caso o ```KNeighborsClassifier``` ainda não foi melhor do que os demais, mas conseguimos ter uma melhora. Em casos reais, uma melhora de 0.05 pode ser a diferença entre ter um algoritmo que a área de negócios não se sente confiante com um outro algoritmo no qual a área de negócio sente-se mais à vontade para trabalhar.